In [1]:
# install dependencies
%pip install openai jinja2 ipywidgets d20 python-dotenv

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
# load dotenv file
import os

from dotenv import load_dotenv
load_dotenv()

if os.environ.get("OPENAI_API_KEY") is None:
    raise Exception("missing OPENAI API KEY...")

In [3]:
# load config file
import tomllib

config_name = "config.toml"
config:dict[str, any] = None
with open(config_name, "rb") as f:
    config = tomllib.load(f)

print(f"{config_name} loaded: v{config["version"]}")

config.toml loaded: v0.0.2


In [4]:
# load config
from jinja2 import BaseLoader, Environment

meta = config.get("meta", None)

messages:dict[str, str] = []
for msg in config["messages"]:
    role = msg.get("role")
    raw_content = msg.get("content")

    print(f"<< {role} >>")
    
    template = Environment(loader=BaseLoader()).from_string(raw_content)
    content = template.render(meta)
    print(content)

    messages.append({"role": role, "content": content})
    print("------")

<< system >>
You are not an AI Model. You are a game master of the table top role-play game: Call of Calthulhu. 

Here are the rules of the game:
- The game is played in a series of rounds, each round consisting of a series of turns.

- You should EVALUATE player's action, if the situation needs a skill check, then respond like this:"""
some context about the skill check...
skill: skill_name (using snake_case, must included in the character sheet)
difficulty: diffuculty_level (easy, medium or hard)
"""

- Otherwise you should respond like this:"""
description of player's current situation...
possible actions:
1. 1st action description...
2. 2nd action description...
3. 3rd action description...
any other possible actions...
"""

- The round will END when ROUND RULES are met, you should summerize the round in the ENDING message like this:"""
some context of the ending...
round summary: the summary of the round...
"""
------
<< system >>
player's CHARACTER SHEET:
- name: Jone Doe
- age: 

In [5]:
# start interactive chat
import re
import functools

import ipywidgets as widgets
from IPython.display import display

from openai import OpenAI
import d20

def count_user_actions():
    count = 0
    for msg in messages:
        if msg["role"] == "user":
            count += 1
    return count

def on_custom_input(b, input:widgets.Text):
    role = "user"
    content = f"{input.value} (TURN: {count_user_actions()+1})"
    messages.append({"role":role, "content":content})
    print(f"USER ACTION: {content}")
    chat()

def on_action(b, index:str):
    role = "user"
    content = f"I have chosen action {index}. (TURN: {count_user_actions()+1})"
    messages.append({"role":role, "content":content})
    print(f"USER ACTION: {content}")
    chat()

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def on_skill(b, skill:str, difficulty:str):
    role = "user"
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. (TURN: {count_user_actions()+1})"
    messages.append({"role":role, "content":content})
    print(f"USER ACTION: {content}")
    chat()

# parse message into key-value pairs
pattern = re.compile(r"^([\w_ *]+)\:[ \s]*(.+)*$")
action_pattern = re.compile(r"(\d+). *(.+)")
def parse_message(content):
    parsed: dict = {}
    current_key = None
    lines = content.splitlines()
    
    # parse the content line by line
    for line in lines:
        res = pattern.search(line)
        if res is not None:
            current_key = res.group(1)
            if res.group(2) is not None:
                parsed[current_key] = res.group(2)
            else:
                parsed[current_key] = ""
        elif current_key is not None:
            parsed[current_key] += "\n" + line
    
    keys = parsed.keys()
    # possible actions
    vbox = widgets.HBox()
    if "possible actions" in keys:
        ma = action_pattern.finditer(parsed["possible actions"])
        for m in ma:
            action_index=m.group(1)
            action_button = widgets.Button(description=action_index)
            action_button.on_click(functools.partial(on_action, index=action_index))
            vbox.children += (action_button,)
        custom_action_button = widgets.Button(description="CUSTOM")
        custom_action_button.on_click(functools.partial(on_custom_action))
        vbox.children += (custom_action_button,)
        display(vbox)
    
    # skill check
    if "skill" in keys and "difficulty" in keys:
        skill = parsed.get("skill")
        difficulty = parsed.get("difficulty")
        skill_button = widgets.Button(description=f"SKILL CHECK - {skill.upper()}", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        display(skill_button)
    
    if "round summary" in keys:
        summary = parsed["round summary"]
        next_round_button = widgets.Button(description="NEXT ROUND")
        display(next_round_button)

def chat():
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    stream = client.chat.completions.create(
        model=os.environ.get("OPENAI_MODEL_NAME"),
        messages=messages,
        stream=True,
        temperature=float(os.environ.get("OPENAI_TEMPERATURE"))
    )
    print("\nASSISTANT:")
    generated = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            delta = chunk.choices[0].delta.content
            print(delta, end="")
            generated += delta
    messages.append({"role": "assistant", "content": generated})
    parse_message(messages[-1]["content"])

print(f"{messages[-1]["role"].upper()}:")
print(messages[-1]["content"])
# parse the last message
parse_message(messages[-1]["content"])

ASSISTANT:
round name: The Haunted Mansion
round context: You have been hired by a wealthy businessman to investigate a haunted mansion in the outskirts of Arkham.
The mansion is said to be cursed and anyone who enters it never returns.
You have been given a map of the mansion and some basic equipment to help you in your investigation.
Your goal is to find out the truth behind the curse and put an end to it.

possible actions:
1. Enter the mansion.
2. Take some photos of the mansion's exterior.
3. Talk to the butler. (may need a persuasion skill check)


USER ACTION: I have chosen action 1. (TURN: 1)

ASSISTANT:
As you push open the heavy, ornate front door of the mansion, an eerie silence greets you. The grand entrance hall is dimly lit, dust motes dancing in the slivers of light that pierce through the grimy windows. The air smells musty, a testament to years of neglect. Ahead of you, a grand staircase spirals upwards, while doors on either side promise hidden secrets within the mansion's depths.

possible actions:
1. Explore the room to the left of the entrance.
2. Head towards the room on the right.
3. Ascend the grand staircase to investigate the upper floors.
4. Examine the portraits and artifacts in the entrance hall more closely.

USER ACTION: I have chosen action 1. (TURN: 2)

ASSISTANT:
You cautiously open the door to the left of the entrance and find yourself in what appears to be a library. Shelves upon shelves of books line the walls, their titles obscured by layers of dust. A large desk sits in the center of the room, papers and photographs scattered across its surface. The air is thick with the scent of old leather and paper. It's quiet here, the only sound being the occasional creak of the mansion settling around you.

possible actions:
1. Examine the books on the shelves for any titles that stand out.
2. Search the desk for any clues or interesting documents.
3. Take photos of the room, focusing on the desk and any unusual features.
4. Leave the library and head back to the entrance hall to explore another area.

USER ACTION: I have chosen action 4. (TURN: 3)

ASSISTANT:
Returning to the entrance hall, you find yourself once again at the crossroads of your investigation within the mansion. The eerie silence seems to press in from the walls, and the dim light casts long shadows, making the grand space feel both imposing and unwelcoming.

possible actions:
1. Head towards the room on the right of the entrance.
2. Ascend the grand staircase to investigate the upper floors.
3. Examine the portraits and artifacts in the entrance hall more closely.
4. Try to find a way to the basement.

USER ACTION: I have chosen action 1. (TURN: 4)

ASSISTANT:
As you enter the room on the right, you're immediately struck by its stark contrast to the library you just left. This appears to be a dining room, with a long table set for what must have been a grand feast, now left to rot. The silverware is tarnished, and the china is cracked. As you take a step forward, a sudden movement catches your eye—a large, shadowy figure darts across the room and disappears. Upon closer inspection, you realize it was a massive, emaciated dog that seemed more ghost than flesh. Its eyes held a haunting glow as it vanished through the far wall.

This sight might shake even the sturdiest of minds.

skill: psychology
difficulty: medium

Button(description='SKILL CHECK - PSYCHOLOGY', layout=Layout(width='auto'), style=ButtonStyle())

USER ACTION: I am making a skill check using PSYCHOLOGY against a difficulty of medium.
And I rolled a 54 for a result of Failure. (TURN: 5)

ASSISTANT:
The sight of the spectral dog darting through the room and vanishing into the wall sends a shiver down your spine, unsettling you deeply. Despite your attempts to rationalize the encounter, your mind reels at the impossibility of what you've just witnessed. The experience leaves you feeling anxious and on edge, casting a shadow over your confidence as you proceed. The mansion's eerie atmosphere seems to thicken, playing tricks on your senses and making you question what is real and what might be a figment of your overactive imagination.

With this turn, you've reached the end of the round without entering the basement of the mansion. However, as you stand in the dining room, trying to collect your thoughts and shake off the unease, a low, ominous sound echoes from beneath the mansion, sending a chill through the air. It's unclear what 

Button(description='NEXT ROUND', style=ButtonStyle())